In [1]:
import requests
from IPython.display import JSON
import csv, os, json
from datetime import datetime

In [2]:
api_endpoint = "https://staging.harritydev.com/api/uspto/applications/"
#analyze_oa_url = "https://staging.harritydev.com/api/nlp/api/v1/parser/analyze/office-action"
analyze_oa_url = "http://localhost:5000/api/v1/parser/analyze/office-action"
root_folder = "data/oa_samples"

staging_bearer_token = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJlbWFpbCI6ImFiYWxhYmFuQGhhcnJpdHlkZXYub25taWNyb3NvZnQuY29tIiwiZmlyc3ROYW1lIjoiQW5kcmlpIiwibGFzdE5hbWUiOiJCYWxhYmFuIE9mZmljZSAzNjUiLCJwZXJtaXNzaW9ucyI6WyJwb3J0Zm9saW8iLCJ1c2VycyIsIm9yZ2FuaXphdGlvbnMiLCJzcGVjaWZpY2F0aW9ucyIsImZvcm1zIiwidGVtcGxhdGVzIiwiYXBwbGljYXRpb25zIiwidXNwdG8iLCJtZXNzYWdlcyIsIndvcmtmbG93cyIsImFkbWluaXN0cmF0aW9uIiwiZG9ja2V0aW5nX3Rhc2tzIiwiYWRtaW5pc3RyYXRpb25fdGFza3MiLCJib3Rfc2NyaXB0cyJdLCJ0ZWFtcyI6WyJEcmFmdGluZyBUZWFtIl0sIm1zU2VjdXJpdHlHcm91cHMiOlt7ImlkIjoiODY3NmJjY2MtZjMzYi00YTUyLTg2NGYtZjA0ZDM1NjI5Zjc1Iiwic2VjdXJpdHlJZGVudGlmaWVyIjoiUy0xLTEyLTEtMjI1NTkyODUyNC0xMjQ2OTUwMjAzLTEzMDc1OTQ2MzAtMTk3MzM3OTYzNyIsImRpc3BsYXlOYW1lIjoiU29tZU90aGVyR3JvdXAiLCJvblByZW1pc2VzTmFtZSI6bnVsbH0seyJpZCI6ImNiYTA4ODdkLWQzYTMtNGY4ZS04YWI3LWM1NWQ3YWRkMmZiZSIsInNlY3VyaXR5SWRlbnRpZmllciI6IlMtMS0xMi0xLTM0MTYyOTU1NDktMTMzNDc2MDM1NS0xNTczMjM4NjY2LTMxOTA4MDc5MzAiLCJkaXNwbGF5TmFtZSI6IkFub3RoZXJHcm91cCIsIm9uUHJlbWlzZXNOYW1lIjpudWxsfV0sImlhdCI6MTY4NDkzODk1NiwiZXhwIjoxNjg3NTMwOTU2LCJzdWIiOiI2MmQ5ODA0Y2U5MjMyN2FiZjljN2QxMTYifQ.qI5Pfh_rwwFJszRBAahYucbN3C3g-1ABQBe27gzyWPo"
dev_bearer_token = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJlbWFpbCI6ImFiYWxhYmFuQGhhcnJpdHlkZXYub25taWNyb3NvZnQuY29tIiwiZmlyc3ROYW1lIjoiQW5kcmlpIiwibGFzdE5hbWUiOiJCYWxhYmFuIE9mZmljZSAzNjUiLCJwZXJtaXNzaW9ucyI6WyJwb3J0Zm9saW8iLCJ1c2VycyIsIm9yZ2FuaXphdGlvbnMiLCJzcGVjaWZpY2F0aW9ucyIsImZvcm1zIiwidGVtcGxhdGVzIiwiYXBwbGljYXRpb25zIiwidXNwdG8iLCJtZXNzYWdlcyIsIndvcmtmbG93cyIsImFkbWluaXN0cmF0aW9uIiwiZG9ja2V0aW5nX3Rhc2tzIiwiYWRtaW5pc3RyYXRpb25fdGFza3MiLCJib3Rfc2NyaXB0cyJdLCJ0ZWFtcyI6WyJEcmFmdGluZyBUZWFtIl0sIm1zU2VjdXJpdHlHcm91cHMiOlt7ImlkIjoiODY3NmJjY2MtZjMzYi00YTUyLTg2NGYtZjA0ZDM1NjI5Zjc1Iiwic2VjdXJpdHlJZGVudGlmaWVyIjoiUy0xLTEyLTEtMjI1NTkyODUyNC0xMjQ2OTUwMjAzLTEzMDc1OTQ2MzAtMTk3MzM3OTYzNyIsImRpc3BsYXlOYW1lIjoiU29tZU90aGVyR3JvdXAiLCJvblByZW1pc2VzTmFtZSI6bnVsbH0seyJpZCI6ImNiYTA4ODdkLWQzYTMtNGY4ZS04YWI3LWM1NWQ3YWRkMmZiZSIsInNlY3VyaXR5SWRlbnRpZmllciI6IlMtMS0xMi0xLTM0MTYyOTU1NDktMTMzNDc2MDM1NS0xNTczMjM4NjY2LTMxOTA4MDc5MzAiLCJkaXNwbGF5TmFtZSI6IkFub3RoZXJHcm91cCIsIm9uUHJlbWlzZXNOYW1lIjpudWxsfV0sImlhdCI6MTY4NDkzODk1NiwiZXhwIjoxNjg3NTMwOTU2LCJzdWIiOiI2MmQ5ODA0Y2U5MjMyN2FiZjljN2QxMTYifQ.qI5Pfh_rwwFJszRBAahYucbN3C3g-1ABQBe27gzyWPo"

# Create the headers dictionary with the authorization header
headers = {
    "Authorization": f"Bearer {staging_bearer_token}"
}

os.makedirs(root_folder, exist_ok=True)

In [3]:
def load_application(application):
    try:
        response = requests.get(api_endpoint+application, headers=headers)
        response.raise_for_status()

        return response.json()["documents"]

    except Exception as e:
        return None
        # swallow

In [4]:
def download_file(name):
    try:
        response = requests.get(api_endpoint+application+"/documents/" + 
                                name, headers=headers)
        response.raise_for_status()

    except requests.exceptions.HTTPError as e:
        print(f"An HTTP error occurred: {e}")
    
    return response.content

In [5]:
def save_binary(folder, name, content):
    with open(folder + "/" + name, "wb") as f:
        f.write(content)

In [6]:
def save_pdf(folder, name, content):
    with open(folder + "/" + name + ".pdf", "wb") as f:
        f.write(content)

In [7]:
def get_previous_claims(claims, oa):
    threshold_date = datetime.strptime(oa["date"], "%Y-%m-%dT%H:%M:%S.%fZ")
    sorted_claims = sorted(claims, key=lambda x: datetime.strptime(x["date"],
                            "%Y-%m-%dT%H:%M:%S.%fZ"), reverse=True)
    for clm in sorted_claims:
        clm_date = datetime.strptime(clm["date"], "%Y-%m-%dT%H:%M:%S.%fZ")
        if clm_date < threshold_date:
            return clm

In [8]:
def store_application_documents(application, documents):
    OAs = [doc for doc in documents if doc["code"]=="CTNF" or doc["code"]=="CTFR"]
    CLMs = [doc for doc in documents if doc["code"]=="CLM"]
    for OA in OAs:
        try:
            oa_date_str = OA["date"].split('T')[0]
            output_folder = (root_folder + "/" + application + "-" + oa_date_str + "-" + OA["code"])
            os.makedirs(output_folder, exist_ok=True)

            oapdf = download_file(OA["id"] + ".pdf")
            save_binary(output_folder, oa_date_str + "-" + OA["code"] + ".pdf", oapdf)
            
            CLM = get_previous_claims(CLMs, OA)
            
            oa_file = download_file(OA["id"] + ".xml")
            save_binary(output_folder, oa_date_str + "-" + OA["code"] + ".xml", oa_file)
            
            clm = download_file(CLM["id"] + ".xml")
            save_binary(output_folder, CLM["date"].split('T')[0] + "-" + CLM["code"] + ".xml", clm)
            
            print(f'{OA["code"]} for {application} downloaded')
        except Exception as e:
            print(f'Failed to download OA for {application}: {e}')

In [12]:
applications = [];

with open('data/allowed_2022-2023.csv') as csvfile:
    app_reader = csv.DictReader(csvfile, delimiter=',')
    for row in app_reader:
        application = row['Id']
        try:
            documents = load_application(application)
            if documents:
                store_application_documents(application, documents)
            else:
                print(f'{application} does not have files')
        except Exception as e:
            print(f'Failed to get documents for {application}: {e}')

17823152 does not have files
17820633 does not have files
17852776 does not have files
17808720 does not have files
17805964 does not have files
17805889 does not have files
17805602 does not have files
CTNF for 17805599 downloaded
CTNF for 17661996 downloaded
CTNF for 17661598 downloaded
CTNF for 17659450 downloaded
CTNF for 17658700 downloaded
CTNF for 17693613 downloaded
CTNF for 17652615 downloaded
CTNF for 17669676 downloaded
CTNF for 17649995 downloaded
CTNF for 17591714 downloaded
CTNF for 17587096 downloaded
CTNF for 17587037 downloaded
CTNF for 17648966 downloaded
CTNF for 17580840 downloaded
CTNF for 17644398 downloaded
CTNF for 17644400 downloaded
CTNF for 17540875 downloaded
CTNF for 17455469 downloaded
CTNF for 17453666 downloaded
CTNF for 17518108 downloaded
CTNF for 17453124 downloaded
CTNF for 17452851 downloaded
CTNF for 17513010 downloaded
CTNF for 17510199 downloaded
CTNF for 17451093 downloaded
CTNF for 17501898 downloaded
CTNF for 17450867 downloaded
CTNF for 17500